<a href="https://colab.research.google.com/github/lupis30puc/yelp_bert_random_forest/blob/update-6/RF_mimic_BERT_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier

import time
import joblib
import torch

## Initializing the values for the model
x_train, x_test, y_train, y_test

In [ ]:
x_train = pd.read_pickle('/content/drive/MyDrive/Yelp/model_128_/binary_train_rf_feb_02')
x_test = pd.read_pickle('/content/drive/MyDrive/Yelp/model_128_/binary_test_rf_feb_02')

In [ ]:
y_train = torch.load('/content/drive/MyDrive/Yelp/model_128_/pred_labels_train')
y_test = torch.load('/content/drive/MyDrive/Yelp/model_128_/pred_labels_test')

## Grid search
I performed a grid search to determined the best parameters for the Random Forest model, with a cross validation of 10. 

Taking into account only the number of estimators (trees) and the minimum sample leaves for each category.

It takes 6h 28 min aprox.

In [ ]:
param_grid = {'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'n_estimators': [200, 300, 1000] }
# The parameter grid to explore, as a dictionary mapping estimator parameters to sequences of allowed values.

In [ ]:
rmfr = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator = rmfr, param_grid = param_grid, cv = 10)
# Exhaustive search over specified parameter values for an estimator.
# The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

In [ ]:
%%time
grid_search.fit(x_train, y_train) # Run fit with all sets of parameters.
#rmfr.fit(x_train,y_train)

CPU times: user 6h 27min 55s, sys: 5.57 s, total: 6h 28min
Wall time: 6h 28min 38s


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                 

In [ ]:
best_model = grid_search.best_params_  # Parameter setting that gave the best results on the hold out data.
print(best_model)

{'min_samples_leaf': 2, 'n_estimators': 1000}


In [ ]:
#saving the best model
random_f = rf_grid.best_estimator_ 

In [ ]:
rf_grid.best_score_ 

0.8926297475383134

In [ ]:
# for the best estimator on the grid search
print("Test  Accuracy : %.2f"%random_f.score(x_test, y_test))
print("Train Accuracy : %.2f"%random_f.score(x_train, y_train))

Test  Accuracy : 0.90
Train Accuracy : 0.98


In [ ]:
%%time
print("Random Forest 1000 estimators, min leaf 2")
predrf_g = random_f.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrf_g))
print("Score:",round(accuracy_score(y_test,predrf_g)*100,2))
print("Classification Report:")
print(classification_report(y_test,predrf_g))

Random Forest 1000 estimators, min leaf 2
Confusion Matrix for Random Forest Classifier:
[[1170  111]
 [ 149 1169]]
Score: 90.0
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1281
           1       0.91      0.89      0.90      1318

    accuracy                           0.90      2599
   macro avg       0.90      0.90      0.90      2599
weighted avg       0.90      0.90      0.90      2599

CPU times: user 1.77 s, sys: 0 ns, total: 1.77 s
Wall time: 1.78 s


In [ ]:
# saving the grid search
joblib.dump(grid_search, "/content/drive/MyDrive/Yelp/model_99/rf_grid_search.joblib", compress=3)

['/content/drive/MyDrive/Yelp/model_99/rf_grid_search.joblib']

In [ ]:
#make a graph were the test and train accuracy are closer to each other, if they cross or are very close 